In [1]:
import sent2vec
filename = '../resources/BioSentVec.bin'
model = sent2vec.Sent2vecModel()
try:
	model.load_model(filename)
except Exception as e:
	print(e)
print('model successfully loaded')

model successfully loaded


In [2]:
from biosentvec_embeds_fxn import run_me

In [3]:
import csv
import biosentvec_embeds_fxn
import biosentvec_embeds_tfidf_fxn
import xlsxwriter
import numpy as np
import random

import re

In [4]:
in_f = '../resources/artificial_indications_filled_v2.csv'
reader = csv.reader(open(in_f))
easy_queries = []
hard_queries = []
for row in reader:
    if row[2] != 'narr_names': # Don't include header row
        [_,name,narr_name, simple, difficult, _, _] = row
        if simple != '':
            easy_queries.append([simple, '', '', ''])
        if difficult != '':
            hard_queries.append([difficult, '', '', ''])

In [56]:
## Code to find what words of queries are present in narr title

from nltk import word_tokenize
from abbreviations import solve_abbvs_fast

in_f = '../resources/artificial_indications_filled_v2.csv'
reader = csv.reader(open(in_f))
easy_has_narr_name = []
hard_has_narr_name = []
for row in reader:
    if row[2] != 'narr_names': # Don't include header row
        [_,name,narr_name, simple, difficult, _, _] = row
        
        narr_tkns = [x.lower() for x in word_tokenize(narr_name)]
        simp_tkns = [x.lower() for x in word_tokenize(simple)] + solve_abbvs_fast(simple, verbose=False)
        print(simp_tkns)
        diff_tkns = [x.lower() for x in word_tokenize(difficult)] + solve_abbvs_fast(difficult, verbose=False)
        print(diff_tkns)
        
        if simple != '':
            easy_has_narr_name.append(sum([1 for narr_tkn in narr_tkns if narr_tkn in simp_tkns]) / len(simp_tkns))
                
        if difficult != '':
            hard_has_narr_name.append(sum([1 for narr_tkn in narr_tkns if narr_tkn in diff_tkns])  / len(diff_tkns))

print(sum(easy_has_narr_name)/205,sum(hard_has_narr_name)/205)

['screen', 'for', 'breast', 'cancer', 'in', '52', 'yo', 'female', 'year', 'old']
['genetic', 'screening', 'shows', 'brca1', 'mutation', '.', 'high', 'risk', 'screening', 'breast', 'cancer', '1']
['firm', 'mass', 'in', 'left', 'breast', ',', 'pregnant', ',', 'screen', 'for', 'malignancy']
['firm', 'mass', 'in', 'left', 'breast', 's/p', 'hip', 'surgery', ',', 'history', 'of', 'diabetes', 'and', 'hypertension', ',', 'recent', 'pregnancy', ',', 'normal', 'lactation']
['s/p', 'silicone', 'breast', 'implantation']
['s/p', 'right', 'mastectomy', 'and', 'bilateral', 'breast', 'implant', 'reconstruction', ',', 'check', 'for', 'potential', 'rupture', 'status', 'post']
['focal', 'right', 'breast', 'pain']
['strong', 'family', 'history', 'of', 'breast', 'cancer', ',', 'planning', 'for', 'bilateral', 'mastectomy', '.', 'evaluate', 'for', 'intermittent', 'mild', 'right', 'breast', 'pain', 'at', 'site', 'of', 'surgery']
['right', 'breast', 'nipple', 'discharge']
['right', 'breast', 'white', 'non-spon

['hand', 'pain', 'after', 'fall']
['10yo', 'with', 'wrist', 'pain', 'and', 'swelling', 'after', 'falling', 'while', 'playing', 'soccer', '.', 'no', 'external', 'bleeding', '.']
['hip', 'pain', 'after', 'ground-level', 'fall']
['80yo', 'woman', 'with', 'history', 'of', 'hypertension', 'presenting', 'with', 'left', 'hip', 'pain', 'after', 'ground-level', 'fall']
['suspected', 'ankle', 'sprain']
['18yo', 'male', 'with', 'ankle', 'swelling', 'and', 'pain', 'after', 'playing', 'basketball', '.', 'able', 'to', 'walk', 'independently', 'but', 'with', 'pain', '.', 'ankle', 'is', 'tender', '.']
['nonpenetrating', 'foot', 'injury', 'suspected', 'fracture']
['40yo', 'with', '8/10', 'dorsal', 'left', 'foot', 'pain', 'after', 'dropping', 'tool', 'on', 'foot', '.', 'not', 'able', 'to', 'put', 'weight', 'on', 'left', 'foot', '.', 'suspected', 'fracture', '.']
['knee', 'injury', 'after', 'fall', 'suspected', 'pcl', 'tear', 'posterior', 'cruciate', 'ligament']
['patient', 'with', 'knee', 'pain', 'after

['child', 'with', 'hip', 'click', 'and', 'swayback', ',', 'suspected', 'hip', 'developmental', 'dysplasia']
['5', 'month', 'old', 'female', 'with', 'right', 'hip', 'pain', 'and', 'swaying', ',', 'concerning', 'for', 'congential', 'hip', 'dislocation']
['child', 'with', 'fever', 'for', '2', 'weeks', 'with', 'no', 'apparent', 'source']
['2', 'month', 'old', 'child', 'presenting', 'with', 'fever', 'of', '39c', 'for', 'past', '3', 'days', ',', 'with', 'leukocytosis', 'and', 'no', 'respiratory', 'symptoms', '.']
['child', 'with', 'altered', 'mental', 'status', 'after', 'ground', 'level', 'fall', ',', 'suspected', 'head', 'trauma']
['22', 'month', 'old', 'child', 'presenting', 'with', 'somnolescence', 'after', '6', 'foot', 'fall', 'and', 'loss', 'of', 'consciousness', 'for', '10', 'seconds', '.', 'suspected', 'head', 'injury', '.']
['child', 'with', 'sudden', 'severe', 'headache', 'and', 'nausea']
['9yo', 'with', 'acute', 'onset', ',', 'progressive', 'headache', 'after', '3', 'weeks', 'of', 

0.3125932603981384 0.09791242284081243


In [ ]:
import importlib
# importlib.reload(biosentvec_embeds_fxn)
importlib.reload(biosentvec_embeds_tfidf_fxn)

In [5]:
easy_results_doc, easy_results_head, easy_results_tf, easy_results = biosentvec_embeds_tfidf_fxn.run_me(model, easy_queries)
hard_results_doc, hard_results_head, hard_results_tf, hard_results = biosentvec_embeds_tfidf_fxn.run_me(model, hard_queries)



0it [00:00, ?it/s]

docs and headers successfully embedded
Processing query 0
VERBOSE: False
ABV [['year', 'old']]
FLTR ['screen', 'breast', 'cancer', 'yo', 'female']
embedding:  ['year', 'old'] 0.2857142857142857


1it [00:02,  2.29s/it]

Processing query 1
VERBOSE: False
ABV []
FLTR ['firm', 'mass', 'left', 'breast', 'pregnant', 'screen', 'malignancy']


2it [00:02,  1.69s/it]

...

203it [00:51,  3.66it/s]

Processing query 203
VERBOSE: False
ABV [['coronary', 'artery', 'bypass', 'graft'], ['deep', 'vein', 'thrombosis']]
FLTR ['patient', 'lung', 'cancer', 'remission', 'recent', 'cabg', 'surgery', 'treated', 'pe', 'recovery', 'dvt', 'shown', 'bilateral', 'lower', 'extremity', 'plan', 'place', 'permanent', 'inferior', 'vena', 'cava', 'filter']
embedding:  ['coronary', 'artery', 'bypass', 'graft'] 0.15384615384615385
embedding:  ['deep', 'vein', 'thrombosis'] 0.12


204it [00:51,  3.74it/s]

Processing query 204
VERBOSE: False
ABV []
FLTR ['patient', 'thoracic', 'endovascular', 'repair', 'aortic', 'root', 'aneurysm', 'year', 'ago', 'presenting', 'trend', 'mild', 'aneurysm', 'size', 'increase']


205it [00:51,  3.97it/s]


In [6]:
names = np.load('../resources/names.npy')


In [ ]:
tf_weights = [0,0.25,0.5,0.75,1,1.5,2,3,5,7,10,15,20]
tf_files = [20,75,100]
tuning_res_2 = []
for tf_file in tf_files:
    for each_weight in tf_weights:
        easy_results_doc, easy_results_head, easy_results_tf, easy_results = biosentvec_embeds_tfidf_fxn.run_me(model, easy_queries, each_weight, tf_file = '../resources/tfidf_embs_%d.npy' % tf_file)
        top_3, not_top_3, accu = calc_performance(easy_results)
        tuning_res_2.append([0,each_weight,tf_file,top_3,not_top_3,accu])

        hard_results_doc, hard_results_head, hard_results_tf, hard_results = biosentvec_embeds_tfidf_fxn.run_me(model, hard_queries, each_weight)
        top_3, not_top_3, accu = calc_performance(hard_results)
        tuning_res_2.append([1,each_weight,tf_file,top_3,not_top_3,accu])



In [ ]:
tuning_res

In [ ]:
def calc_performance(results):
    top_3 = []
    not_top_3 = []
    for i,result in enumerate(results):
    #     print(i)
        res_files = [result[0][0]]
        pos = 1
    #     print(res_files)
        while pos < len(result):
            if result[pos][0] != result[pos-1][0]:
                res_files.append(result[pos][0])
            pos = pos+1

        this_narr_name = get_narr_name(names[i])

        if this_narr_name in res_files[0:3]:
    #         print('YES', this_narr_name, res_files.index(this_narr_name))
            top_3.append([this_narr_name, res_files.index(this_narr_name),i])
        else:
    #         print('NO', this_narr_name, res_files.index(this_narr_name))
            not_top_3.append([this_narr_name, res_files.index(this_narr_name),i])

    print()
    return (len(top_3)), (len(not_top_3)), (len(top_3)/(len(top_3)+len(not_top_3)))

In [8]:
np.save('../resources/artificial_inds_easy_key_v6.npy',easy_results)
np.save('../resources/artificial_inds_hard_key_v6.npy',hard_results)
# easy_results = np.load('../resources/artificial_inds_easy_key.npy')
# hard_results = np.load('../resources/artificial_inds_hard_key.npy')

In [7]:
def get_narr_name(fName):
	narr_path = '../resources/criteria_links.txt'
	narr_f = open(narr_path, 'r', encoding ='utf8')

	f_name = fName.split('.')
	f_name = f_name[0].split('_')
	
	mtch = re.search(r'Category '+f_name[0]+r':.+\n(\t.+\n)*?\tDisease '+f_name[1]+r': (.+)',narr_f.read())
	if mtch:
		return(mtch.group(2))
	else:
		print(fName, 'You messed up in the narrative name!')
		return(None)

In [ ]:
# top_cos = []
# not_top_cos = []
# for each in hard_top_3:
#     top_cos.append(hard_results[each[2]][0][4])
# for i,each in enumerate(hard_not_top_3):
#     not_top_cos.append(hard_results[each[2]][0][4])

# print(top_cos)
# print(not_top_cos)
# print(np.mean(top_cos))
# print(np.mean(not_top_cos))

In [ ]:
easy_doc_top_3 = []
easy_doc_not_top_3 = []
for i,result in enumerate(easy_results_doc):
#     print(i)
    res_files = [result[0][0]]
    pos = 1
#     print(res_files)
    while pos < len(result):
        if result[pos][0] != result[pos-1][0]:
            res_files.append(result[pos][0])
        pos = pos+1
    
    this_narr_name = get_narr_name(names[i])
    

    if this_narr_name in res_files[0:3]:
#         print('YES', this_narr_name, res_files.index(this_narr_name))
        easy_doc_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    else:
#         print('NO', this_narr_name, res_files.index(this_narr_name))
        easy_doc_not_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    
print()
print(len(easy_doc_top_3))
print(len(easy_doc_not_top_3))

In [ ]:
easy_head_top_3 = []
easy_head_not_top_3 = []
for i,result in enumerate(easy_results_head):
#     print(i)
    res_files = [result[0][0]]
    pos = 1
#     print(res_files)
    while pos < len(result):
        if result[pos][0] != result[pos-1][0]:
            res_files.append(result[pos][0])
        pos = pos+1
    
    this_narr_name = get_narr_name(names[i])
    

    if this_narr_name in res_files[0:3]:
#         print('YES', this_narr_name, res_files.index(this_narr_name))
        easy_head_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    else:
#         print('NO', this_narr_name, res_files.index(this_narr_name))
        easy_head_not_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    
print()
print(len(easy_head_top_3))
print(len(easy_head_not_top_3))

In [ ]:
easy_tf_top_3 = []
easy_tf_not_top_3 = []
for i,result in enumerate(easy_results_tf):
#     print(i)
    res_files = [result[0][0]]
    pos = 1
#     print(res_files)
    while pos < len(result):
        if result[pos][0] != result[pos-1][0]:
            res_files.append(result[pos][0])
        pos = pos+1
    
    this_narr_name = get_narr_name(names[i])
    

    if this_narr_name in res_files[0:3]:
#         print('YES', this_narr_name, res_files.index(this_narr_name))
        easy_tf_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    else:
#         print('NO', this_narr_name, res_files.index(this_narr_name))
        easy_tf_not_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    
print()
print(len(easy_tf_top_3))
print(len(easy_tf_not_top_3))

In [28]:
easy_top_3 = []
easy_not_top_3 = []
for i,result in enumerate(easy_results):
#     print(i)
    res_files = [result[0][0]]
    pos = 1
#     print(res_files)
    while pos < len(result):
        if result[pos][0] != result[pos-1][0]:
            res_files.append(result[pos][0])
        pos = pos+1
    
    this_narr_name = get_narr_name(names[i])
    

    if this_narr_name in res_files[0:3]:
#         print('YES', this_narr_name, res_files.index(this_narr_name))
        easy_top_3.append([this_narr_name, res_files.index(this_narr_name),i, names[i]])
    else:
#         print('NO', this_narr_name, res_files.index(this_narr_name))
        easy_not_top_3.append([this_narr_name, res_files.index(this_narr_name),i, names[i]])
    
print()
print(len(easy_top_3))
print(len(easy_not_top_3))
print(len(easy_top_3)/(len(easy_top_3)+len(easy_not_top_3)))


201
4
0.9804878048780488


In [ ]:
easy_not_top_3

In [14]:
# For plotting the graph!!

for x in range(1,189):
    easy_top_3 = []
    easy_not_top_3 = []
    for i,result in enumerate(easy_results):
    #     print(i)
        res_files = [result[0][0]]
        pos = 1
    #     print(res_files)
        while pos < len(result):
            if result[pos][0] != result[pos-1][0]:
                res_files.append(result[pos][0])
            pos = pos+1

        this_narr_name = get_narr_name(names[i])


        if this_narr_name in res_files[0:x]:
    #         print('YES', this_narr_name, res_files.index(this_narr_name))
            easy_top_3.append([this_narr_name, res_files.index(this_narr_name),i, names[i]])
        else:
    #         print('NO', this_narr_name, res_files.index(this_narr_name))
            easy_not_top_3.append([this_narr_name, res_files.index(this_narr_name),i, names[i]])
    
    print(len(easy_top_3)/(len(easy_top_3)+len(easy_not_top_3)))

0.8048780487804879
0.9365853658536586
0.9804878048780488
0.9902439024390244
0.9902439024390244
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1

In [13]:
# For plotting the graph!!
for x in range(1,189):
    hard_top_3 = []
    hard_not_top_3 = []
    for i,result in enumerate(hard_results):
    #     print(i)
        res_files = [result[0][0]]
        pos = 1
    #     print(res_files)
        while pos < len(result):
            if result[pos][0] != result[pos-1][0]:
                res_files.append(result[pos][0])
            pos = pos+1

        this_narr_name = get_narr_name(names[i])


        if this_narr_name in res_files[0:x]:
    #         print('YES', this_narr_name, res_files.index(this_narr_name))
            hard_top_3.append([this_narr_name, res_files.index(this_narr_name),i])
        else:
    #         print('NO', this_narr_name, res_files.index(this_narr_name))
            hard_not_top_3.append([this_narr_name, res_files.index(this_narr_name),i])
    
    print(len(hard_top_3)/(len(hard_top_3)+len(hard_not_top_3)))

0.624390243902439
0.7853658536585366
0.848780487804878
0.8731707317073171
0.8878048780487805
0.9219512195121952
0.926829268292683
0.9365853658536586
0.9463414634146341
0.9512195121951219
0.9609756097560975
0.9658536585365853
0.9707317073170731
0.975609756097561
0.9804878048780488
0.9804878048780488
0.9853658536585366
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.9951219512195122
0.99512195121951

In [ ]:
hard_doc_top_3 = []
hard_doc_not_top_3 = []
for i,result in enumerate(hard_results_doc):
#     print(i)
    res_files = [result[0][0]]
    pos = 1
#     print(res_files)
    while pos < len(result):
        if result[pos][0] != result[pos-1][0]:
            res_files.append(result[pos][0])
        pos = pos+1
    
    this_narr_name = get_narr_name(names[i])
    

    if this_narr_name in res_files[0:3]:
#         print('YES', this_narr_name, res_files.index(this_narr_name))
        hard_doc_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    else:
#         print('NO', this_narr_name, res_files.index(this_narr_name))
        hard_doc_not_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    
print()
print(len(hard_doc_top_3))
print(len(hard_doc_not_top_3))

In [ ]:
hard_head_top_3 = []
hard_head_not_top_3 = []
for i,result in enumerate(hard_results_head):
#     print(i)
    res_files = [result[0][0]]
    pos = 1
#     print(res_files)
    while pos < len(result):
        if result[pos][0] != result[pos-1][0]:
            res_files.append(result[pos][0])
        pos = pos+1
    
    this_narr_name = get_narr_name(names[i])
    

    if this_narr_name in res_files[0:3]:
#         print('YES', this_narr_name, res_files.index(this_narr_name))
        hard_head_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    else:
#         print('NO', this_narr_name, res_files.index(this_narr_name))
        hard_head_not_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    
print()
print(len(hard_head_top_3))
print(len(hard_head_not_top_3))

In [ ]:
hard_tf_top_3 = []
hard_tf_not_top_3 = []
for i,result in enumerate(hard_results_tf):
#     print(i)
    res_files = [result[0][0]]
    pos = 1
#     print(res_files)
    while pos < len(result):
        if result[pos][0] != result[pos-1][0]:
            res_files.append(result[pos][0])
        pos = pos+1
    
    this_narr_name = get_narr_name(names[i])
    

    if this_narr_name in res_files[0:3]:
#         print('YES', this_narr_name, res_files.index(this_narr_name))
        hard_tf_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    else:
#         print('NO', this_narr_name, res_files.index(this_narr_name))
        hard_tf_not_top_3.append([this_narr_name, res_files.index(this_narr_name)])
    
print()
print(len(hard_tf_top_3))
print(len(hard_tf_not_top_3))

In [19]:
hard_top_3 = []
hard_not_top_3 = []
for i,result in enumerate(hard_results):
#     print(i)
    res_files = [result[0][0]]
    pos = 1
#     print(res_files)
    while pos < len(result):
        if result[pos][0] != result[pos-1][0]:
            res_files.append(result[pos][0])
        pos = pos+1
    
    this_narr_name = get_narr_name(names[i])

    if this_narr_name in res_files[0:3]:
#         print('YES', this_narr_name, res_files.index(this_narr_name))
        hard_top_3.append([this_narr_name, res_files.index(this_narr_name),i, names[i]])
    else:
#         print('NO', this_narr_name, res_files.index(this_narr_name))
        hard_not_top_3.append([this_narr_name, res_files.index(this_narr_name),i, names[i]])
    
print()
print(len(hard_top_3))
print(len(hard_not_top_3))
print(len(hard_top_3)/(len(hard_top_3)+len(hard_not_top_3)))


174
31
0.848780487804878


In [30]:
hard_not_top_3

[['Chest Pain-Possible Acute Coronary Syndrome', 3, 13, '02_05.txt'],
 ['Dyspnea-Suspected Cardiac Origin', 6, 16, '02_08.txt'],
 ['Known or Suspected Congenital Heart Disease in the Adult',
  4,
  18,
  '02_10.txt'],
 ['Suspected Infective Endocarditis', 11, 20, '02_12.txt'],
 ['Suspected Pulmonary Embolism', 7, 22, '02_14.txt'],
 ['Clinically Suspected Adnexal Mass, No Acute Symptoms', 17, 45, '04_05.txt'],
 ['Female Infertility', 13, 46, '04_06.txt'],
 ['First Trimester Vaginal Bleeding', 3, 47, '04_07.txt'],
 ['Gestational Trophoblastic Disease', 17, 48, '04_08.txt'],
 ['Chronic Extremity Joint Pain–Suspected Inflammatory Arthritis',
  10,
  68,
  '06_09.txt'],
 ['Chronic Hip Pain', 14, 70, '06_11.txt'],
 ['Primary Bone Tumors', 16, 80, '06_21.txt'],
 ['Shoulder Pain-Atraumatic', 3, 81, '06_22.txt'],
 ['Soft-Tissue Masses', 54, 83, '06_24.txt'],
 ['Suspected Osteomyelitis, Septic Arthritis, or Soft Tissue Infection (Excluding Spine and Diabetic Foot)',
  12,
  86,
  '06_27.txt'],
 

In [15]:
import numpy as np

# print(np.mean([x[1] for x in easy_doc_top_3] + [x[1] for x in easy_doc_not_top_3]))
# print(np.std([x[1] for x in easy_doc_top_3] + [x[1] for x in easy_doc_not_top_3]))
# print()
# print(np.mean([x[1] for x in easy_head_top_3] + [x[1] for x in easy_head_not_top_3]))
# print(np.std([x[1] for x in easy_head_top_3] + [x[1] for x in easy_head_not_top_3]))
# print()
print(np.mean([x[1] for x in easy_top_3] + [x[1] for x in easy_not_top_3]))
print(np.std([x[1] for x in easy_top_3] + [x[1] for x in easy_not_top_3]))
print()
# print(np.mean([x[1] for x in hard_doc_top_3] + [x[1] for x in hard_doc_not_top_3]))
# print(np.std([x[1] for x in hard_doc_top_3] + [x[1] for x in hard_doc_not_top_3]))
# print()
# print(np.mean([x[1] for x in hard_head_top_3] + [x[1] for x in hard_head_not_top_3]))
# print(np.std([x[1] for x in hard_head_top_3] + [x[1] for x in hard_head_not_top_3]))
# print()
print(np.mean([x[1] for x in hard_top_3] + [x[1] for x in hard_not_top_3]))
print(np.std([x[1] for x in hard_top_3] + [x[1] for x in hard_not_top_3]))

0.35609756097560974
1.3416496544776155

1.6585365853658536
4.794355197610689


In [29]:
# Subgroup analysis

easy_subgroup = {}
for each_res in easy_top_3:
    cat = each_res[-1].split('_')[0]
    if cat not in easy_subgroup.keys():
        easy_subgroup[cat] = [1,0]
    else:
        easy_subgroup[cat][0] += 1
for each_res in easy_not_top_3:
    cat = each_res[-1].split('_')[0]
    if cat not in easy_subgroup.keys():
        easy_subgroup[cat] = [0,1]
    else:
        easy_subgroup[cat][1] += 1

print(easy_subgroup)
for key,each in easy_subgroup.items():
    print(key,each[0],each[1])

hard_subgroup = {}
for each_res in hard_top_3:
    cat = each_res[-1].split('_')[0]
    if cat not in hard_subgroup.keys():
        hard_subgroup[cat] = [1,0]
    else:
        hard_subgroup[cat][0] += 1
for each_res in hard_not_top_3:
    cat = each_res[-1].split('_')[0]
    if cat not in hard_subgroup.keys():
        hard_subgroup[cat] = [0,1]
    else:
        hard_subgroup[cat][1] += 1

for key,each in hard_subgroup.items():
    print(key,each[0],each[1])

{'01': [9, 0], '02': [14, 0], '03': [18, 0], '04': [17, 1], '05': [1, 0], '06': [27, 1], '07': [22, 2], '08': [16, 0], '09': [16, 0], '10': [20, 0], '11': [23, 0], '12': [18, 0]}
01 9 0
02 14 0
03 18 0
04 17 1
05 1 0
06 27 1
07 22 2
08 16 0
09 16 0
10 20 0
11 23 0
12 18 0
01 9 0
02 9 5
03 18 0
04 14 4
05 1 0
06 22 6
07 23 1
08 14 2
09 11 5
10 18 2
11 19 4
12 16 2


In [ ]:
hard_results[156]

In [ ]:
res_doc, res_head, res = biosentvec_embeds_fxn.run_me(model,[['35yo woman with recent abnormal periods presents with menometrorrhagia. MRI shows uterine fibroids. She prefers to bear children in the future. Next steps?','','','']])

In [ ]:
for each in res[0]:
    print(each)
len(res_doc)
# a_res_2 = [x[1] for x in a_res[0]]

In [11]:
hard_not_top_3

[['Chest Pain-Possible Acute Coronary Syndrome', 3, 13, '02_05.txt'],
 ['Dyspnea-Suspected Cardiac Origin', 6, 16, '02_08.txt'],
 ['Known or Suspected Congenital Heart Disease in the Adult',
  4,
  18,
  '02_10.txt'],
 ['Suspected Infective Endocarditis', 11, 20, '02_12.txt'],
 ['Suspected Pulmonary Embolism', 7, 22, '02_14.txt'],
 ['Clinically Suspected Adnexal Mass, No Acute Symptoms', 17, 45, '04_05.txt'],
 ['Female Infertility', 13, 46, '04_06.txt'],
 ['First Trimester Vaginal Bleeding', 3, 47, '04_07.txt'],
 ['Gestational Trophoblastic Disease', 17, 48, '04_08.txt'],
 ['Chronic Extremity Joint Pain–Suspected Inflammatory Arthritis',
  10,
  68,
  '06_09.txt'],
 ['Chronic Hip Pain', 14, 70, '06_11.txt'],
 ['Primary Bone Tumors', 16, 80, '06_21.txt'],
 ['Shoulder Pain-Atraumatic', 3, 81, '06_22.txt'],
 ['Soft-Tissue Masses', 54, 83, '06_24.txt'],
 ['Suspected Osteomyelitis, Septic Arthritis, or Soft Tissue Infection (Excluding Spine and Diabetic Foot)',
  12,
  86,
  '06_27.txt'],
 

In [ ]:
# tf_idf = np.load('../resources/tfidf_idf.npy')
# tf_words = np.load('../resources/tfidf_words.npy')
# tf_arr_np = np.load('../resources/tfidf_arrs_full.npy',allow_pickle=True).item()
import numpy as np
full = np.load('../../resources/tfidf.npy')
doc_embs = []
for doc in full:
    print(doc[0][0])
    emb = np.zeros(700)
    for xx in range(50):
        res = np.squeeze(model.embed_sentence(doc[xx][1]))
        emb = np.add(emb, res)

    emb = emb/50
    doc_embs.append(emb)

len(doc_embs)
len(doc_embs[0])

In [ ]:
np.save('../../resources/tfidf_embs_50.npy',doc_embs)

In [3]:
import numpy as np
full = np.load('../resources/tfidf.npy')
len(full[0])

200

In [ ]:
import biosentvec_embeds_tfidf_fxn

In [ ]:
import importlib
importlib.reload(biosentvec_embeds_tfidf_fxn)

In [ ]:
res_doc, res_head, res_tf, res = biosentvec_embeds_tfidf_fxn.run_me(model,[['59yo with history of hypertension and IBD presents with fever, ruq pain, high suspicion of sclerosing cholangitis. next steps','','','']])

In [ ]:
for each in res[0]:
    print(each)
len(res_doc)